# Zemel et al. pre-processing fairness intervention

Zemel et al. (2013) proposes a clustering method which transforms the original data set by expressing points as linear combinations of learnt cluster centres. The transformed data set is as close as possible to the original while containing as little information as possible about the sensitive attributes. Thereby, demographic parity is achieved.

The output of their method includes besides a fair data representation also fair label predictions, which allows the comparison according to the usual fairness metrics. We apply their approach as implemented by IBM's AIF360 fairness tool box.

In [ ]:
from pathlib import Path

import joblib
import pandas as pd
import plotly.graph_objs as go
from aif360.algorithms.preprocessing.lfr import LFR  # noqa
from aif360.datasets import StandardDataset
from helpers.fairness_measures import accuracy, disparate_impact_d

In [ ]:
from helpers import export_plot

## Load data

We have committed preprocessed data to the repository for reproducibility and we load it here. Check out the preprocessing notebook for details on how this data was obtained.

In [ ]:
artifacts_dir = Path("../../../artifacts")

In [ ]:
# override data_dir in source notebook
# this is stripped out for the hosted notebooks
artifacts_dir = Path("../../../../artifacts")

In [ ]:
data_dir = artifacts_dir / "data" / "recruiting"

In [ ]:
train = pd.read_csv(data_dir / "processed" / "train.csv")
val = pd.read_csv(data_dir / "processed" / "val.csv")
test = pd.read_csv(data_dir / "processed" / "test.csv")

AIF360 requires expressing the original data sets via the "StandardDataset" class.

In [ ]:
train_sds = StandardDataset(
    train,
    label_name="employed_yes",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
test_sds = StandardDataset(
    test,
    label_name="employed_yes",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
val_sds = StandardDataset(
    val,
    label_name="employed_yes",
    favorable_classes=[1],
    protected_attribute_names=["race_white"],
    privileged_classes=[[1]],
)
index = train_sds.feature_names.index("race_white")

In [ ]:
privileged_groups = [{"race_white": 1.0}]
unprivileged_groups = [{"race_white": 0.0}]

## Learn fair representation

We chose the hyperparameters $A_x, A_y, A_z$ and $k$ by a grid search, and load a pretrained model from disk for reproducibility, however we encourage you to experiment with other values of these hyperparameters. 

In [ ]:
TR = joblib.load(artifacts_dir / "models" / "recruiting" / "zemel.pkl")

# TR = LFR(
#     unprivileged_groups=unprivileged_groups,
#     privileged_groups=privileged_groups,
#     k=5,
#     Ax=0.01,
#     Ay=1.0,
#     Az=1500.0,
# )
# TR = TR.fit(train_sds)  # , maxiter=500, maxfun=500)

Apply transformation to validation data

In [ ]:
transf_val_sds = TR.transform(val_sds)

In [ ]:
acc = accuracy(transf_val_sds.labels.flatten(), val.employed_yes)
print("Accuracy after fairness intervention =", acc)

Evaluate fairness and accuracy

In [ ]:
# Fair labels
val_fair_labels = transf_val_sds.labels.flatten()

In [ ]:
print("Accuracy =", accuracy(val_fair_labels, val.employed_yes))
print(
    "Black accuracy =",
    accuracy(
        val_fair_labels[val.race_white == 0],
        val.employed_yes[val.race_white == 0],
    ),
)
print(
    "White accuracy =",
    accuracy(
        val_fair_labels[val.race_white == 1],
        val.employed_yes[val.race_white == 1],
    ),
)
print(
    "Mean black score =", val_fair_labels[val.race_white == 0].mean(),
)
print(
    "Mean white score =", val_fair_labels[val.race_white == 1].mean(),
)

dp_d = disparate_impact_d(val_fair_labels, val.race_white)
print("Sex demographic parity =", dp_d)

We visualise the difference in mean outcomes using a bar chart.

In [ ]:
dp_bar = go.Figure(
    data=[
        go.Bar(
            x=[race],
            y=[val_fair_labels[val.race_white == race].mean()],
            name="White" if race else "Black",
        )
        for race in range(2)
    ],
    layout={"yaxis": {"range": [0, 1]}},
)
dp_bar

In [ ]:
export_plot(dp_bar, "zemel-dp.json")